In [1]:
from pandas_data_handler import download_csv_to_dataframe
from lookups import CsvUrlTweets,CsvUrlCoinsInfo,CoinsEnergyConsumption,CsvUrlHistoricalData

# Example usage:
df = download_csv_to_dataframe(CsvUrlHistoricalData.CRYPTO_TWEETS_V2)


In [2]:
import pandas as pd
from numpy import NaN
import nltk
import re
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from yahoofinancials import YahooFinancials


In [3]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [4]:
def preprocess_text(text):
    if isinstance(text, str):
        # remove punctuation and special characters
        text = re.sub(r'[^\w\s]', '', text)
        # convert to lowercase
        text = text.lower()
        # tokenize text
        tokens = nltk.word_tokenize(text)
        # remove stop words
        tokens = [token for token in tokens if token not in stop_words]
        # lemmatize text
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        # join tokens back into text
        text = ' '.join(tokens)
    return text


In [5]:
df = df.dropna(subset=['full_text'])
df['full_text'] = df['full_text'].fillna('')
df['full_text'] = df['full_text'].str.lower()
df['clean_text'] = df['full_text'].apply(preprocess_text)

In [6]:
df['importance_coefficient'] = df['retweet_count'] + 2 * df['favorite_count'] + 0.5 * df['reply_count']
# # Find the minimum and maximum values of the importance coefficient
min_value = df['importance_coefficient'].min()
max_value = df['importance_coefficient'].max()

# # Normalize the importance coefficient
df['importance_coefficient_normalized'] = (df['importance_coefficient'] - min_value) / (max_value - min_value)
# # Sort the DataFrame based on the "created_at" column in ascending order
df = df.sort_values('created_at', ascending=True)

In [7]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
def analyze_sentiment(df):
    # Create a copy of the input DataFrame
    df_selected = df.copy()

    # Initialize the VADER sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # Calculate sentiment scores and add them to the DataFrame
    df_selected['scores'] = df_selected['full_text'].apply(lambda description: sid.polarity_scores(description))
    df_selected['compound'] = df_selected['scores'].apply(lambda score_dict: score_dict['compound'])
    
    # Create a new column for sentiment_type and classify based on the compound score
    df_selected['sentiment_type'] = df_selected['compound'].apply(lambda avg_compound: 'POSITIVE' if avg_compound > 0 else 'NEUTRAL' if avg_compound == 0 else 'NEGATIVE')

    return df_selected
sentiment_df=analyze_sentiment(df)
sentiment_df

,created_at,favorite_count,full_text,reply_count,retweet_count,user_id,new_coins,clean_text,importance_coefficient,importance_coefficient_normalized,scores,compound,sentiment_type
Unnamed: 0,,,,,,,,,,,,,
32666,2021-02-01,154,#privacy is a human right. learn how to make y...,18,23,aantonop,(bitcoin),privacy human right learn make bitcoin transac...,340.0,0.000588,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL
29639,2021-02-01,17,"overall btc trading volume has increased, but ...",1,5,CoinDeskData,(btc),overall btc trading volume increased average t...,39.5,0.000068,"{'neg': 0.0, 'neu': 0.95, 'pos': 0.05, 'compou...",0.2124,POSITIVE
29613,2021-02-01,3,"on average, the return distribution of btc ske...",0,1,CoinDeskData,(btc),average return distribution btc skews slightly...,7.0,0.000012,"{'neg': 0.053, 'neu': 0.769, 'pos': 0.177, 'co...",0.7010,POSITIVE
39638,2021-02-01,3496,i sent some! https://t.co/mfyrz35zjf\n\nyou sh...,731,686,VitalikButerin,(doge),sent httpstcomfyrz35zjf givedirectly great wor...,8043.5,0.013905,"{'neg': 0.06, 'neu': 0.856, 'pos': 0.084, 'com...",0.2225,POSITIVE
32660,2021-02-01,0,rt @reg_mati: la privacidad es un derecho huma...,0,7,aantonop,(bitcoin),rt reg_mati la privacidad e un derecho humano ...,7.0,0.000012,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,2023-06-12,173,ã°âââ¨the #litecoin halving is in 50 days ã°...,98,31,CryptoTony__,(litecoin),ãâââthe litecoin halving 50 day ãâââ httpstco4...,426.0,0.000736,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL
883,2023-06-12,41,still not personally bidding any alts right no...,13,2,AltcoinSherpa,(btc),still personally bidding alt right dont know g...,90.5,0.000156,"{'neg': 0.094, 'neu': 0.78, 'pos': 0.126, 'com...",0.4295,POSITIVE
826,2023-06-12,193,latest: @sturdyfinance has just been exploited...,116,46,coingecko,(eth),latest sturdyfinance exploited 4426 eth httpst...,490.0,0.000847,"{'neg': 0.25, 'neu': 0.75, 'pos': 0.0, 'compou...",-0.4588,NEGATIVE


In [9]:
# sentiment_df.to_csv("tweet_sentiment_analysis")

In [10]:
def get_historical_prices(tickers, start_date, end_date):
    data = {}
    for ticker in tickers:
        yahoo_financials = YahooFinancials(ticker)
        historical_data = yahoo_financials.get_historical_price_data(start_date, end_date, "daily")
        data[ticker] = historical_data[ticker]['prices']
    dfs = []
    for ticker, prices in data.items():
        df = pd.DataFrame(prices)
        df = df.drop('date', axis=1).set_index('formatted_date')
        df.columns = [f"{ticker}_close", f"{ticker}_high", f"{ticker}_low", f"{ticker}_open", f"{ticker}_volume", f"{ticker}_adjclose"]
        df['formatted_date'] = pd.to_datetime(df.index) # Add formatted_date column
        dfs.append(df)
    merged_df = pd.concat(dfs, axis=1)
    return merged_df

In [11]:
tickers = ['BTC-USD', 'ETH-USD','DOGE-USD']
start_date = '2021-01-02'
end_date = '2023-06-12'

btc_df = get_historical_prices(tickers, start_date, end_date)
# Move formatted_date column to the first position
btc_df = btc_df[["formatted_date"] + [col for col in btc_df.columns if col != "formatted_date"]]
# Delete duplicate formatted_date columns
btc_df = btc_df.loc[:, ~btc_df.columns.duplicated()]
print(btc_df.head())

               formatted_date  BTC-USD_close  BTC-USD_high   BTC-USD_low  \
formatted_date                                                             
2021-01-02         2021-01-02   33155.117188  29091.181641  29376.455078   
2021-01-03         2021-01-03   34608.558594  32052.316406  32129.408203   
2021-01-04         2021-01-04   33440.218750  28722.755859  32810.949219   
2021-01-05         2021-01-05   34437.589844  30221.187500  31977.041016   
2021-01-06         2021-01-06   36879.699219  33514.035156  34013.613281   

                BTC-USD_open  BTC-USD_volume  BTC-USD_adjclose  ETH-USD_close  \
formatted_date                                                                  
2021-01-02      32127.267578     67865420765      32127.267578     786.798462   
2021-01-03      32782.023438     78665235202      32782.023438    1006.565002   
2021-01-04      31971.914062     81163475344      31971.914062    1153.189209   
2021-01-05      33992.429688     67547324782      33992.429688

In [20]:
btc_selected = btc_df.iloc[:, :7]
btc_selected = btc_selected.round(0)
btc_selected['formatted_date'] = btc_selected.index

btc_selected.head()
#---ETH-selecte------------
eth_selected = btc_df.iloc[:, 7:13]
eth_selected = eth_selected.round(0)
eth_selected['formatted_date'] = eth_selected.index

eth_selected
#--doge-selected------------
doge_selected = btc_df.iloc[:, 13:19]
doge_selected['formatted_date'] = doge_selected.index

In [21]:
# Calculate price changes
btc_selected['price_changes'] = btc_selected['BTC-USD_close'].diff()
btc_selected['price_changes'] = btc_selected['price_changes'].apply(lambda x: 'positive' if x > 0 else 'negative' if x < 0 else 'neutral')

# Calculate price changes
eth_selected['price_changes'] = eth_selected['ETH-USD_close'].diff()
eth_selected['price_changes'] = eth_selected['price_changes'].apply(lambda x: 'positive' if x > 0 else 'negative' if x < 0 else 'neutral')
eth_selected
# Calculate price changes
doge_selected['price_changes'] = doge_selected['DOGE-USD_close'].diff()
doge_selected['price_changes'] = doge_selected['price_changes'].apply(lambda x: 'positive' if x > 0 else 'negative' if x < 0 else 'neutral')
doge_selected

,DOGE-USD_close,DOGE-USD_high,DOGE-USD_low,DOGE-USD_open,DOGE-USD_volume,DOGE-USD_adjclose,formatted_date,price_changes
formatted_date,,,,,,,,
2021-01-02,0.013698,0.005584,0.005686,0.010615,3421562680,0.010615,2021-01-02,neutral
2021-01-03,0.013867,0.009409,0.010602,0.009771,2707003608,0.009771,2021-01-03,positive
2021-01-04,0.011421,0.007878,0.009785,0.009767,1372398979,0.009767,2021-01-04,negative
2021-01-05,0.010219,0.008972,0.009767,0.009920,687256067,0.009920,2021-01-05,negative
2021-01-06,0.010854,0.009685,0.009923,0.010465,749915516,0.010465,2021-01-06,positive
...,...,...,...,...,...,...,...,...
2023-06-08,0.068426,0.067154,0.067302,0.067966,167825308,0.067966,2023-06-08,negative
2023-06-09,0.070920,0.067626,0.067963,0.069489,274101529,0.069489,2023-06-09,positive
2023-06-10,0.070591,0.058536,0.069489,0.061860,975736541,0.061860,2023-06-10,negative


In [22]:
#change columns name
def remove_chars_before_underscore(df):
    df.columns = df.columns.str.split('_').str[-1]
remove_chars_before_underscore(btc_selected)
remove_chars_before_underscore(eth_selected)
remove_chars_before_underscore(doge_selected)
btc_selected
eth_selected
doge_selected

,close,high,low,open,volume,adjclose,date,changes
formatted_date,,,,,,,,
2021-01-02,0.013698,0.005584,0.005686,0.010615,3421562680,0.010615,2021-01-02,neutral
2021-01-03,0.013867,0.009409,0.010602,0.009771,2707003608,0.009771,2021-01-03,positive
2021-01-04,0.011421,0.007878,0.009785,0.009767,1372398979,0.009767,2021-01-04,negative
2021-01-05,0.010219,0.008972,0.009767,0.009920,687256067,0.009920,2021-01-05,negative
2021-01-06,0.010854,0.009685,0.009923,0.010465,749915516,0.010465,2021-01-06,positive
...,...,...,...,...,...,...,...,...
2023-06-08,0.068426,0.067154,0.067302,0.067966,167825308,0.067966,2023-06-08,negative
2023-06-09,0.070920,0.067626,0.067963,0.069489,274101529,0.069489,2023-06-09,positive
2023-06-10,0.070591,0.058536,0.069489,0.061860,975736541,0.061860,2023-06-10,negative


In [23]:
eth_selected.to_csv('eth_daily_2021_2023')
btc_selected.to_csv('btc_daily_2021_2023')
doge_selected.to_csv('doge_daily_2021_2023')